In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import ddsp
from thesis import newt
import einops
import numpy as np

from ddsp.colab.colab_utils import specplot
import gin

gin.enter_interactive_mode()

In [ ]:
n_samples = 16000
harm = newt.NEWTHarmonic(n_harmonics=2, n_outputs=3, sample_rate=16000, n_samples=n_samples)

# f0 = tf.zeros((2, 16000)) + 440
f0 = tf.linspace(440, 880, n_samples).reshape((1, -1, 1))
f0 = einops.repeat(f0, "b t 1 -> (b2 b) t 1", b2=2)
exciter = harm(f0)
exciter.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(exciter[0, :100])

In [ ]:
import IPython.display


def play_audio(audio):
    IPython.display.display(IPython.display.Audio(audio, rate=16000))


play_audio(exciter[0, :, 0])

## Control module

In [ ]:
f0 = tf.zeros((2, 16000)) + 440
other_controls = tf.zeros((2, 16000)) + 1

In [ ]:
cm = newt.NEWTControlModule(hidden_size=30, embedding_size=10)
cm(f0, other_controls)

In [ ]:
layer = tf.keras.layers.Activation('identity')
output = layer([-3.0, -1.0, 0.0, 2.0])
# list(output.numpy())
output.numpy()

## NEWT waveshaper

In [ ]:
control_embedding_size = 8
batch_size = 2
control = tf.random.normal((batch_size, 100, control_embedding_size))

In [ ]:

shaper = newt.NEWTWaveshaper(
    n_waveshapers=3,
    control_embedding_size=control_embedding_size,
    shaping_fn_hidden_size=16,
)
y = shaper(exciter, control)
y.shape

In [ ]:
plt.plot(y[0])
play_audio(y[0,:])

In [ ]:
a = tf.linspace(-5, 5, 100)
a = einops.repeat(a, "x -> 1 x n_waveshapers", n_waveshapers=shaper.n_waveshapers)
b = shaper.shaping_fn(a)
#b = einops.rearrange(b, "n_waveshapers samples 1 1 -> n_waveshapers samples")

In [ ]:
plt.plot(b[0])

In [ ]:
shapers = newt.NEWTTrainableWaveshapers(n_waveshapers=4)
a = np.linspace(-3, 3, 100)
a = einops.repeat(a, "x -> 1 x n_waveshapers", n_waveshapers=4)
lookup_table = shapers(a)[0]
plt.plot(lookup_table)

In [ ]:
min_value = -3
max_value = 3
n_buckets = 100
batch_size = 2
n_waveshapers = 4

n_samples = 50
a = np.linspace(-4, 4, n_samples)
a = einops.repeat(a, "x -> b x n_waveshapers", b=2, n_waveshapers=n_waveshapers)

a = tf.clip_by_value(a, min_value, max_value)

#b = tf.gather()
indices = tf.round((n_buckets - 1) * (a + max_value) / (max_value - min_value)).astype(tf.int32)
x = einops.repeat(tf.range(n_waveshapers), "n_waveshapers -> b x n_waveshapers", b=batch_size, x=n_samples)
indices = tf.stack([indices, x], axis=-1)

In [ ]:
out_cached = tf.gather_nd(params=lookup_table, indices=indices)
out_cached.shape

In [ ]:
plt.plot(out_cached[0])

In [ ]:
tf.gather_nd(params=lookup_table, indices=tf.convert_to_tensor([[[0,0], [1,1]]]))

In [ ]:
indices